In [ ]:
import numpy as np
import tensorflow as tf 
import cv2
from PIL import Image
from datasets import load_dataset

In [ ]:
ds = load_dataset("microsoft/cats_vs_dogs")

In [ ]:
print(ds['train'][0]['image'])
print(ds['train'][0]['labels'])

In [ ]:
ds

In [ ]:
all_images = []
all_labels = []

target_size = (128, 128)

In [ ]:
def make_square(image):
    width, height = image.size

    # If the image is already square, return the original image
    if width == height:
        return image

    # Determine the size of the new square image (it should be the max of width and height)
    new_size = max(width, height)

    # Create a new black image with a square size
    new_image = Image.new("RGB", (new_size, new_size), color=(0, 0, 0))  # Black background

    # Calculate the position to paste the original image (centered)
    paste_position = ((new_size - width) // 2, (new_size - height) // 2)

    # Paste the original image onto the new black square
    new_image.paste(image, paste_position)

    return new_image

In [ ]:

print('iterating through the dataset')
print('processing...')
for row in ds['train']:
    image = row['image']
    
    new_image = make_square(image)
    
    image_resized = new_image.resize(target_size)
    image_np = np.array(image_resized)
    all_images.append(image_np)
    
    label = row['labels']
    all_labels.append(label)
    
print('Done !')

In [ ]:
# check the shape of everything
count = 0
for image in all_images:
    if len(image.shape) != 3 or (len(image.shape) == 3 and image.shape[2] != 3):
        print(f'shape: {image.shape}')
        count += 1
        
print(f'count: {count}')

**Issue**

We have 5 pictures that don't have 3 channels but just one. We have to eliminate those in order to conver all_images from a list to a numpy array

I've tried to eliminated the elements that have the shape == 2 but with the .remove() method it didn't work. So we have to repopulate another list

after a couple of trial there I founded that there are some images where the last axis has 4 channels instead of 3 (I don't know why, i guess it is some alpha value), so I'm gonna get rid of those images.

In [ ]:
temp_list = []
count_list = []
count = 0

for image in all_images:
    if len(image.shape) == 3 and image.shape[2] == 3:
        temp_list.append(image)
    else:
        count_list.append(count)
    count += 1
    
for count in reversed(count_list):
    all_labels.pop(count)
    
all_images = temp_list

In [ ]:
all_images = np.array(all_images)
all_labels = np.array(all_labels)

print(f'shape images np array: {all_images.shape}')
print(f'shape labels np array: {all_labels.shape}')

In [ ]:
# saving the two arrays into files
import os

folder_name = '../npy_file'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

np.save('../npy_file/images.npy', all_images)
np.save('../npy_file/labels.npy', all_labels)

In [ ]:
image = Image.fromarray(all_images[0])
#image.show()

Create the model

In [ ]:
all_images = all_images/255.0

input_shape = (128, 128, 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size = 0.3, random_state = 42)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(Conv2D(input_shape=input_shape, filters=16, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
batch = 32
epochs = 10

steps_per_epoch = int(np.ceil(len(X_train)/batch))
validation_steps = int(np.ceil(len(X_test)/batch))

folder_name = '../ai_models'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
 
best_model_file = os.path.join(folder_name, 'cat_dog_squared_10.keras')

best_model = ModelCheckpoint(best_model_file, monitor = 'val_accuracy', verbose = 1, save_best_only = True)

history = model.fit(X_train, y_train,
                    batch_size = batch,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_test, y_test),
                    validation_steps = validation_steps,
                    steps_per_epoch = steps_per_epoch,
                    shuffle = True,
                    callbacks = [best_model])

In [ ]:

history.history